<a href="https://colab.research.google.com/github/matssonalex/CIG/blob/main/Cell_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and training

In [5]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Copying the python files from the dictionary to the content dictionary

In [2]:
!cp /content/drive/MyDrive/Colab\ Notebooks/Cell_tracking_challenge/GAN.py /content
!cp /content/drive/MyDrive/Colab\ Notebooks/Cell_tracking_challenge/training.py /content
!cp /content/drive/MyDrive/Colab\ Notebooks/Cell_tracking_challenge/loader.py /content

Copying the numpy files from the dictionary to the content dictionary

In [3]:
!cp /content/drive/MyDrive/Colab\ Notebooks/Cell_tracking_challenge/raw_images.npy /content
!cp /content/drive/MyDrive/Colab\ Notebooks/Cell_tracking_challenge/label_images.npy /content

Training the GAN model

In [8]:
from cProfile import label
from GAN import GAN
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.metrics import mean_absolute_error as mae

def train_GAN(model, batch_size, n_epochs):
#     # hittils endast discriminator som tränas när man kör model.training_step()
    label_img = np.load('label_images.npy')
    raw_img = np.load('raw_images.npy')

#     n_images = len(label_img)
#     for i in range(n_epochs):
#         #for j in range(int(n_images/batch_size))

#         indexes = np.random.randint(0, n_images, (batch_size))
    model.fit(label_img, raw_img, epochs=n_epochs, batch_size=batch_size)

    #data = [label_img[indexes], raw_img[indexes]]
#         loss = model.train_step(data)
#         d_loss = loss['d_loss']
#         g_loss = loss['g_loss']
#         # print('hej')
#         #d_loss
#         print(f'discriminator loss: {d_loss.numpy()}')
#         print(f'generator loss: {g_loss.numpy()}')

        # test_picture = label_img[0]
        #a, b = model.predict(test_picture)
        # plt.imshow(tf.reshape(b[0], (256,256)))
        # plt.show()


def loss_fn_gen(z_label, z_output, pred_fake):
    gamma = 0.8
    mean_abs_err = tf.keras.losses.mean_absolute_error(z_label, z_output)
    return gamma*tf.reshape(mean_abs_err, (10,1)) + (1 - pred_fake**2)


def loss_fn_disc(pred_real, pred_fake):
    return pred_fake**2 + (1 - pred_real)**2


if __name__ == '__main__':
    gan = GAN()
    gan.compile(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn_g=loss_fn_gen,
    loss_fn_d=loss_fn_disc
    )
    train_GAN(gan, 10, 10)


    

Epoch 1/10
96/96 [==============================] - 56s 501ms/step - d_loss: 0.1274 - g_loss: 0.9702
Epoch 2/10
96/96 [==============================] - 48s 501ms/step - d_loss: 0.0108 - g_loss: 0.9951
Epoch 3/10
96/96 [==============================] - 48s 500ms/step - d_loss: 9.3877e-05 - g_loss: 1.0000
Epoch 4/10
96/96 [==============================] - 48s 501ms/step - d_loss: 1.8238e-07 - g_loss: 1.0000
Epoch 5/10
96/96 [==============================] - 48s 500ms/step - d_loss: 1.4851e-07 - g_loss: 1.0000
Epoch 6/10
96/96 [==============================] - 48s 500ms/step - d_loss: 1.2270e-07 - g_loss: 1.0000
Epoch 7/10
96/96 [==============================] - 48s 499ms/step - d_loss: 1.0202e-07 - g_loss: 1.0000
Epoch 8/10
96/96 [==============================] - 48s 501ms/step - d_loss: 8.5658e-08 - g_loss: 1.0000
Epoch 9/10
96/96 [==============================] - 48s 500ms/step - d_loss: 7.2960e-08 - g_loss: 1.0000
Epoch 10/10
96/96 [==============================] - 48s 500ms/

# Plots and Results

In [16]:
li = np.load('label_images.npy')
nli = gan(li[0,:,:])
nli = np.asarray(nli)
ili = Image.fromarray(nli.reshape(256, 256))
ili.show()

TypeError: ignored